In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/csqa-dataset/__huggingface_repos__.json
/kaggle/input/csqa-dataset/csqa_single_stage_generated.csv


In [2]:
df = pd.read_csv('/kaggle/input/csqa-dataset/csqa_single_stage_generated.csv')

In [3]:
df.head()

,question,correct_combinations,incorrect_combinations,notes_private
0,The sanctions against the school were a punish...,"['ignore AND/OR undermine', 'diminish AND/OR n...","['enforce AND/OR avoid', 'authoritarian AND/OR...",Correct options merge the idea of disregarding...
1,Sammy wanted to go to where the people were. ...,"['city center AND/OR public park', 'shopping m...","['race track AND/OR apartment', 'the desert AN...",Correct options merge populated areas like cit...
2,To locate a choker not located in a jewelry bo...,"['jewelry store AND jewelry store staff', 'jew...","['neck AND jewelry box', 'jewelry box AND bout...",Correct options merge 'jewelry store' with add...
3,Google Maps and other highway and street GPS s...,['digital navigation tools AND/OR traditional ...,['united states highways AND/OR digital naviga...,Correct options integrate the transition from ...
4,"The fox walked from the city into the forest, ...","['natural habitat AND/OR food sources', 'shelt...","['pretty flowers AND/OR dense forest', 'hen ho...",Correct combinations merge facts about the fox...


In [4]:
# !pip install -q transformers accelerate bitsandbytes peft trl datasets

In [5]:
import os
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

os.environ['HF_TOKEN'] = user_secrets.get_secret("hugging_face")

In [6]:
from huggingface_hub import login
login(new_session=False)

In [7]:
# from transformers import AutoModelForCausalLM, AutoTokenizer
# from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# MODEL_NAME = "meta-llama/Llama-3.1-8B-Instruct"

# # Load tokenizer
# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# tokenizer.pad_token = tokenizer.eos_token

# # Load model in 4-bit for QLoRA
# model = AutoModelForCausalLM.from_pretrained(
#     MODEL_NAME,
#     load_in_4bit=True,              # 4-bit quantization
#     device_map="auto",              # Automatically split layers to GPU
#     bnb_4bit_quant_type="nf4",      # NormalFloat4 (better for QLoRA)
#     bnb_4bit_use_double_quant=True, # More compression
#     torch_dtype="auto"
# )

# # Prepare for LoRA fine-tuning
# model = prepare_model_for_kbit_training(model)

# # Configure LoRA
# lora_config = LoraConfig(
#     r=8,            # Rank
#     lora_alpha=16,  # Scaling factor
#     target_modules=["q_proj", "v_proj"],  # Apply LoRA to attention layers
#     lora_dropout=0.05,
#     bias="none",
#     task_type="CAUSAL_LM"
# )

# # Wrap model with LoRA
# model = get_peft_model(model, lora_config)

# print("Model ready for QLoRA fine-tuning!")

In [8]:
import random
import ast
import pandas as pd
import numpy as np


def safe_eval(val):
    if isinstance(val, list):  # already list
        return val
    if pd.isna(val):  # NaN
        return []
    if isinstance(val, str):  # string that looks like a list
        return ast.literal_eval(val)
    return []  # fallback

df['correct_combinations'] = df['correct_combinations'].apply(safe_eval)
df['incorrect_combinations'] = df['incorrect_combinations'].apply(safe_eval)


qa_pairs = []
correct_label_cycle = ['A', 'B', 'C', 'D']
label_index = 0  # To cycle through correct answer positions


for idx, row in df.iterrows():
    if len(row['correct_combinations']) == 0 or len(row['incorrect_combinations']) == 0:
        continue  # skip if missing answers
    
    question = row['question']
    # if len(row['correct_combinations']) > 1:
    #     correct_ans = row['correct_combinations'][1]
    # else:
    correct_ans = row['correct_combinations'][0]
    /
    incorrect_list = row['incorrect_combinations']
    if len(incorrect_list) >= 3:
        incorrect_ans = random.sample(incorrect_list, 3)
    else:
        incorrect_ans = random.choices(incorrect_list, k=3)
    
    # We'll fix the correct answer position according to label_index
    labels = ['A', 'B', 'C', 'D']
    correct_label = correct_label_cycle[label_index % 4]
    label_index += 1
    
    # Place correct answer in the position indicated by correct_label
    options = incorrect_ans.copy()
    options.insert(labels.index(correct_label), correct_ans)
    
    qa_pairs.append({
        'question': question,
        'A': options[0],
        'B': options[1],
        'C': options[2],
        'D': options[3],
        'correct_label': correct_label,
        'correct_answer_text': correct_ans
    })

qa_df = pd.DataFrame(qa_pairs)
print(f"Generated {len(qa_df)} QA pairs with balanced correct answer positions.")

qa_df.to_csv('csqa_pairs.csv',index=False)

Generated 5000 QA pairs with balanced correct answer positions.


In [9]:
qa_df.head()

,question,A,B,C,D,correct_label,correct_answer_text
0,The sanctions against the school were a punish...,ignore AND/OR undermine,authoritarian AND/OR yell at,enforce AND/OR avoid,yell at AND/OR diminish,A,ignore AND/OR undermine
1,Sammy wanted to go to where the people were. ...,race track AND/OR apartment,city center AND/OR public park,the desert AND/OR roadblock,roadblock AND/OR public park,B,city center AND/OR public park
2,To locate a choker not located in a jewelry bo...,neck AND jewelry box,jewelry box AND boutique staff,jewelry store AND jewelry store staff,boutique AND jewelry display,C,jewelry store AND jewelry store staff
3,Google Maps and other highway and street GPS s...,united states highways AND/OR digital navigati...,countryside routes AND/OR traditional atlases,oceans navigation AND/OR real-time traffic upd...,digital navigation tools AND/OR traditional at...,D,digital navigation tools AND/OR traditional at...
4,"The fox walked from the city into the forest, ...",natural habitat AND/OR food sources,pretty flowers AND/OR dense forest,storybook AND/OR safety from humans,hen house AND/OR natural habitat,A,natural habitat AND/OR food sources


In [10]:
def replace(string):
    return string.replace('AND/OR','AND')

In [11]:
qa_df.A = qa_df.A.apply(replace)
qa_df.B = qa_df.B.apply(replace)
qa_df.C = qa_df.C.apply(replace)
qa_df.D = qa_df.D.apply(replace)
qa_df.correct_answer_text = qa_df.correct_answer_text.apply(replace)

In [12]:
qa_df.head()

,question,A,B,C,D,correct_label,correct_answer_text
0,The sanctions against the school were a punish...,ignore AND undermine,authoritarian AND yell at,enforce AND avoid,yell at AND diminish,A,ignore AND undermine
1,Sammy wanted to go to where the people were. ...,race track AND apartment,city center AND public park,the desert AND roadblock,roadblock AND public park,B,city center AND public park
2,To locate a choker not located in a jewelry bo...,neck AND jewelry box,jewelry box AND boutique staff,jewelry store AND jewelry store staff,boutique AND jewelry display,C,jewelry store AND jewelry store staff
3,Google Maps and other highway and street GPS s...,united states highways AND digital navigation ...,countryside routes AND traditional atlases,oceans navigation AND real-time traffic updates,digital navigation tools AND traditional atlases,D,digital navigation tools AND traditional atlases
4,"The fox walked from the city into the forest, ...",natural habitat AND food sources,pretty flowers AND dense forest,storybook AND safety from humans,hen house AND natural habitat,A,natural habitat AND food sources


In [13]:
# import json

# def one_shot_one_depth_reasoning(model, tokenizer, qa_dataframe):
#     results = []
#     required_cols = ['question', 'A', 'B', 'C', 'D', 'correct_label']
#     if not all(col in qa_dataframe.columns for col in required_cols):
#         raise ValueError(f"DataFrame must contain the following columns: {required_cols}")

#     valid_choices = {'A', 'B', 'C', 'D'}

#     for idx, row in qa_dataframe.iterrows():
#         predicted_label = None
#         try:
#             # Construct the prompt
#             prompt = f"""
#             You are a logical reasoning AI designed to evaluate multi-factor, multiple-choice questions. To find the correct answer, you must analyze each option and determine if both parts of the "AND" combination positively contribute to the overall goal of the question.
#             Your task is to analyze each option and provide a structured, multi-step breakdown. For each option, you will:
            
#             1. Start with a Hypothesis that reflects the overall goal of the question.
            
#             2. Break down the hypothesis into two sub-hypotheses (SubH 1 and SubH 2) that correspond to the two factors mentioned in the question.
            
#             3. For each sub-hypothesis, provide a Premise and a Conclusion that fully and accurately explains the reasoning.
            
#             4. Finally, provide an AND Logic Evaluation that summarizes how the two parts of the option work together, and a concluding Therefore statement that definitively judges the option's effectiveness.

#             5. At the end, provide the right choice (just the letter and not the full sentence).
            
#             Below is an example for your reference:
            
#             Question: What are ways someone can get warm?
#             A. Wearing thick clothing and turning off the lights
#             B. Drinking hot tea and sitting near an open window
#             C. Turning on a heater and closing windows
#             D. Eating spicy food and taking a cold shower

#             Output: {{
#                 "output_breakdown": [
#                     {{
#                         "option": "A",
#                         "hypothesis": "The person stays warm",
#                         "sub_hypotheses": [
#                             {{
#                                 "id": "SubH 1",
#                                 "factor": "Internal heat is retained (via thick clothing)",
#                                 "premise": "Thick clothing traps the body's natural heat by creating a layer of insulation.",
#                                 "conclusion": "Wearing thick clothing effectively helps retain internal heat."
#                             }},
#                             {{
#                                 "id": "SubH 2",
#                                 "factor": "External cold is reduced (via turning off lights)",
#                                 "premise": "Turning off lights does not significantly change external cold exposure.",
#                                 "conclusion": "Turning off lights is an irrelevant action for reducing cold exposure."
#                             }}
#                         ],
#                         "and_logic_evaluation": "Wearing thick clothing helps retain warmth, BUT turning off the lights is irrelevant to the overall goal.",
#                         "therefore": "This combination is not the best choice."
#                     }},
#                     {{
#                         "option": "B",
#                         "hypothesis": "The person stays warm",
#                         "sub_hypotheses": [
#                             {{
#                                 "id": "SubH 1",
#                                 "factor": "Internal heat is retained (via hot tea)",
#                                 "premise": "Drinking a warm liquid like hot tea provides a temporary, slight increase in core body temperature.",
#                                 "conclusion": "Drinking hot tea provides a temporary increase in internal heat."
#                             }},
#                             {{
#                                 "id": "SubH 2",
#                                 "factor": "External cold is reduced (via sitting near an open window)",
#                                 "premise": "Sitting near an open window allows cold air to enter and increases the body's exposure to cold, causing rapid heat loss.",
#                                 "conclusion": "Sitting near an open window increases heat loss and contradicts the goal of reducing cold exposure."
#                             }}
#                         ],
#                         "and_logic_evaluation": "Drinking hot tea provides temporary warmth, BUT sitting near an open window causes significant heat loss.",
#                         "therefore": "This combination is contradictory and not the best choice."
#                     }},
#                     {{
#                         "option": "C",
#                         "hypothesis": "The person stays warm",
#                         "sub_hypotheses": [
#                             {{
#                                 "id": "SubH 1",
#                                 "factor": "Internal heat is retained (via heater)",
#                                 "premise": "A heater generates and adds heat to the indoor environment.",
#                                 "conclusion": "Turning on a heater increases heat retention."
#                             }},
#                             {{
#                                 "id": "SubH 2",
#                                 "factor": "External cold is reduced (via closed windows)",
#                                 "premise": "Closing windows prevents cold air from entering and warm air from escaping.",
#                                 "conclusion": "Closing windows effectively reduces external cold exposure."
#                             }}
#                         ],
#                         "and_logic_evaluation": "A heater provides heat generation AND closed windows prevent heat from escaping. Both actions work together to maximize warmth.",
#                         "therefore": "This combination is the correct choice."
#                     }},
#                     {{
#                         "option": "D",
#                         "hypothesis": "The person stays warm",
#                         "sub_hypotheses": [
#                             {{
#                                 "id": "SubH 1",
#                                 "factor": "Internal heat is retained (via spicy food)",
#                                 "premise": "Eating spicy food induces a warmth sensation but provides no real thermal gain to the body.",
#                                 "conclusion": "Eating spicy food does not effectively help retain internal heat."
#                             }},
#                             {{
#                                 "id": "SubH 2",
#                                 "factor": "External cold is reduced (via a cold shower)",
#                                 "premise": "Taking a cold shower involves direct contact with cold water, which rapidly draws heat away from the body.",
#                                 "conclusion": "Taking a cold shower significantly increases external cold exposure and causes major heat loss."
#                             }}
#                         ],
#                         "and_logic_evaluation": "Eating spicy food provides a minimal, psychological sense of warmth, BUT a cold shower causes significant heat loss.",
#                         "therefore": "This combination is counterproductive and not the best choice."
#                     }}
#                 ],
#                 "final_answer": "C"
#             }}

#             Now your turn:
#             Question:
#             {str(row['question'])}
            
#             Options:
#             A. {str(row['A'])}
#             B. {str(row['B'])}
#             C. {str(row['C'])}
#             D. {str(row['D'])}
            
#             Output: {{
#               "output_breakdown": [
#                 {{
#                   "option": "A",
#                   "hypothesis": "...",
#                   "sub_hypotheses": [
#                     {{
#                       "id": "SubH 1",
#                       "factor": "...",
#                       "premise": "...",
#                       "conclusion": "..."
#                     }},
#                     {{
#                       "id": "SubH 2",
#                       "factor": "...",
#                       "premise": "...",
#                       "conclusion": "..."
#                     }}
#                   ],
#                   "and_logic_evaluation": "...",
#                   "therefore": "..."
#                 }},
#                 {{
#                   "option": "B",
#                   "hypothesis": "...",
#                   "sub_hypotheses": [
#                     {{
#                       "id": "SubH 1",
#                       "factor": "...",
#                       "premise": "...",
#                       "conclusion": "..."
#                     }},
#                     {{
#                       "id": "SubH 2",
#                       "factor": "...",
#                       "premise": "...",
#                       "conclusion": "..."
#                     }}
#                   ],
#                   "and_logic_evaluation": "...",
#                   "therefore": "..."
#                 }},
#                 {{
#                   "option": "C",
#                   "hypothesis": "...",
#                   "sub_hypotheses": [
#                     {{
#                       "id": "SubH 1",
#                       "factor": "...",
#                       "premise": "...",
#                       "conclusion": "..."
#                     }},
#                     {{
#                       "id": "SubH 2",
#                       "factor": "...",
#                       "premise": "...",
#                       "conclusion": "..."
#                     }}
#                   ],
#                   "and_logic_evaluation": "...",
#                   "therefore": "..."
#                 }},
#                 {{
#                   "option": "D",
#                   "hypothesis": "...",
#                   "sub_hypotheses": [
#                     {{
#                       "id": "SubH 1",
#                       "factor": "...",
#                       "premise": "...",
#                       "conclusion": "..."
#                     }},
#                     {{
#                       "id": "SubH 2",
#                       "factor": "...",
#                       "premise": "...",
#                       "conclusion": "..."
#                     }}
#                   ],
#                   "and_logic_evaluation": "...",
#                   "therefore": "..."
#                 }}
#               ],
#               "final_answer": ""
#             }}
#             """
            
#             inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
#             inputs = {key: value.to(model.device) for key, value in inputs.items()}

#             output_tokens = model.generate(
#                 input_ids=inputs['input_ids'],
#                 attention_mask=inputs['attention_mask'],
#                 max_new_tokens=1024,
#                 do_sample=False
#             )

#             generated_token_ids = output_tokens[0][inputs['input_ids'].shape[1]:]
#             generated_text = tokenizer.decode(generated_token_ids, skip_special_tokens=True)

#             # --- REVISED POST-PROCESSING STEPS ---
#             # 1. Add a placeholder to make the JSON valid if the model's output is incomplete
#             closing_brace_index = generated_text.rfind("}}")
#             if closing_brace_index != -1:
#                 json_string = generated_text[:closing_brace_index+2]
#             else:
#                 json_string = generated_text + "}}" # In case the model's output is completely missing the end

#             # 2. Attempt to parse the JSON output
#             try:
#                 data = json.loads(json_string)
#                 extracted_label = data.get('final_answer')
#                 if extracted_label and extracted_label in valid_choices:
#                     predicted_label = extracted_label
#                 else:
#                     predicted_label = None # Fallback if final_answer is missing or invalid
#             except json.JSONDecodeError:
#                 # Fallback to a less-structured approach if JSON parsing fails
#                 # This could happen if the model's output is malformed
#                 pass
            
#             # The rest of your logging logic remains the same
#             results.append({
#                 'question': str(row['question']),
#                 'options': f"A. {str(row['A'])} B. {str(row['B'])} C. {str(row['C'])} D. {str(row['D'])}",
#                 'reasoning': generated_text,
#                 'generated_label': predicted_label,
#                 'correct_label': str(row['correct_label']).strip().upper()
#             })
            
#         except Exception as e:
#             print(f"Error processing row {idx}: {e}")
#             results.append({
#                 'question': str(row['question']),
#                 'options': f"A. {str(row['A'])} B. {str(row['B'])} C. {str(row['C'])} D. {str(row['D'])}",
#                 'reasoning': "",
#                 'generated_label': None,
#                 'correct_label': str(row['correct_label']).strip().upper()
#             })

#     return results

    

# # Usage example
# results = one_shot_one_depth_reasoning(model,tokenizer, qa_df.iloc[:20])
# my_results = pd.DataFrame(results)
# my_results.to_csv('csqa_oneshot_1d_reasoning.csv',index=False)

# for res in results[:20]:
#     print(f"Q: {res['question']}")
#     print(f"Options: {res['options']}")
#     print(f"Correct answer: {res['correct_label']}")
#     print(f"Reasoning: {res['reasoning']}")
#     print(f"Predicted answer: {res['generated_label']}")
#     print("-----")

In [14]:
import os
import json
import time
import pandas as pd
import random
from openai import OpenAI, RateLimitError
from tenacity import retry, wait_random_exponential, stop_after_attempt
from datasets import load_dataset
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

# --- 1. Setup and API Call Functions ---

openai_key = user_secrets.get_secret("openai")
if not openai_key:
    raise ValueError("OPENAI_API_KEY environment variable not set.")
client = OpenAI(api_key=openai_key)


def call_gpt4o_mini(messages):
    """Sends a prompt to the GPT-4o-mini API with robustness for errors."""
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            response_format={"type": "json_object"},
            temperature=0,
            n=1
        )
        content = response.choices[0].message.content
        return json.loads(content)
    except RateLimitError as e:
        print(f"Rate limit error: {e}. Retrying...")
        raise
    except json.JSONDecodeError as e:
        print(f"Received invalid JSON from API: {e}. Retrying...")
        raise e
    except Exception as e:
        print(f"An unexpected error occurred during the API call: {e}")
        raise e


def one_shot_two_depth_reasoning(row):
    """
    Creates a prompt for a two-depth reasoning task with nested premises.
    
    The prompt instructs the model to analyze each option by:
    1.  Starting with a top-level Hypothesis (H).
    2.  Breaking it down into two Sub-Hypotheses (H1 and H2).
    3.  For each Sub-Hypothesis, analyzing the specific Actions (A1, A2) taken.
    4.  Providing a two-step breakdown for each action: a list of Premises that lead to a single Conclusion.
    5.  Finally, evaluating the AND logic and providing a definitive 'Therefore' statement.
    """
    user_content = f"""
    You are a logical reasoning AI designed to evaluate complex, multi-factor, multiple-choice questions. To find the correct answer, you must analyze each option and determine if both parts of the "AND" combination positively contribute to the overall goal.

    Your task is to analyze each option and provide a structured, multi-step breakdown. For each option, you will:

    1.  Start with a top-level Hypothesis (H) that reflects the overall goal of the question.
    
    2.  Break down the hypothesis into two Sub-Hypotheses (H1 and H2) that correspond to the two factors mentioned in the question.
    
    3.  For each Sub-Hypothesis, analyze the specific Actions (A1, B1, C1, D1 and A2, B2, C2, D2) taken in the option.
    
    4.  Provide a two-step breakdown for each action: a list of Premises that lead to a single Conclusion.
    
    5.  Provide an AND Logic Evaluation that summarizes how the two actions work together.
    
    6.  Finally, provide a concluding Therefore statement that definitively judges the option's effectiveness.

    7.  At the end, provide the right choice (just the letter, e.g., 'A', and not the full sentence).
    
    Below is an example for your reference. The premises for each action are listed and lead to a single conclusion.
    
    Question: What are ways someone can get warm?
    A. Wearing thick clothing and turning off the lights
    B. Drinking hot tea and sitting near an open window
    C. Turning on a heater and closing windows
    D. Eating spicy food and taking a cold shower
    
    Output: {{
        "output_breakdown": [
            {{
                "option": "A",
                "hypothesis": "The person stays warm",
                "sub_hypotheses": [
                    {{
                        "id": "H1",
                        "factor": "Internal heat is generated or retained",
                        "actions": [
                            {{
                                "id": "A1",
                                "description": "The person wears thick clothing",
                                "premises": [
                                    {{
                                        "id": "A1.1",
                                        "text": "Thick clothing acts as insulation by trapping air."
                                    }},
                                    {{
                                        "id": "A1.2",
                                        "text": "Insulation slows down the rate of body heat loss."
                                    }}
                                ],
                                "conclusion": "Therefore, wearing thick clothing effectively helps to retain internal heat."
                            }}
                        ]
                    }},
                    {{
                        "id": "H2",
                        "factor": "Exposure to cold is reduced",
                        "actions": [
                            {{
                                "id": "A2",
                                "description": "The person turns off the lights",
                                "premises": [
                                    {{
                                        "id": "A2.1",
                                        "text": "Turning off lights reduces light levels."
                                    }},
                                    {{
                                        "id": "A2.2",
                                        "text": "Light levels have a negligible effect on a person's body temperature."
                                    }}
                                ],
                                "conclusion": "Therefore, turning off lights is irrelevant for reducing cold exposure."
                            }}
                        ]
                    }}
                ],
                "and_logic_evaluation": "Wearing thick clothing effectively helps retain warmth, BUT turning off the lights is an irrelevant action for the goal.",
                "therefore": "This combination is not the best choice."
            }},
            {{
                "option": "B",
                "hypothesis": "The person stays warm",
                "sub_hypotheses": [
                    {{
                        "id": "H1",
                        "factor": "Internal heat is generated or retained",
                        "actions": [
                            {{
                                "id": "B1",
                                "description": "The person drinks hot tea",
                                "premises": [
                                    {{
                                        "id": "B1.1",
                                        "text": "Drinking hot tea provides a temporary source of internal warmth."
                                    }},
                                    {{
                                        "id": "B1.2",
                                        "text": "Warm liquids can slightly increase core body temperature."
                                    }}
                                ],
                                "conclusion": "Therefore, drinking hot tea provides a temporary increase in internal heat."
                            }}
                        ]
                    }},
                    {{
                        "id": "H2",
                        "factor": "Exposure to cold is reduced",
                        "actions": [
                            {{
                                "id": "B2",
                                "description": "The person sits near an open window",
                                "premises": [
                                    {{
                                        "id": "B2.1",
                                        "text": "Open windows expose the person to cold air."
                                    }},
                                    {{
                                        "id": "B2.2",
                                        "text": "Exposure to cold air causes rapid heat loss."
                                    }}
                                ],
                                "conclusion": "Therefore, sitting near an open window increases heat loss and contradicts the goal of staying warm."
                            }}
                        ]
                    }}
                ],
                "and_logic_evaluation": "Drinking hot tea provides temporary warmth, BUT sitting near an open window causes significant heat loss.",
                "therefore": "This combination is contradictory and not the best choice."
            }},
            {{
                "option": "C",
                "hypothesis": "The person stays warm",
                "sub_hypotheses": [
                    {{
                        "id": "H1",
                        "factor": "Internal heat is generated or retained",
                        "actions": [
                            {{
                                "id": "C1",
                                "description": "The person turns on a heater",
                                "premises": [
                                    {{
                                        "id": "C1.1",
                                        "text": "Heaters convert electrical energy into thermal energy."
                                    }},
                                    {{
                                        "id": "C1.2",
                                        "text": "Thermal energy raises the ambient temperature of the room."
                                    }}
                                ],
                                "conclusion": "Therefore, turning on a heater increases heat retention."
                            }}
                        ]
                    }},
                    {{
                        "id": "H2",
                        "factor": "Exposure to cold is reduced",
                        "actions": [
                            {{
                                "id": "C2",
                                "description": "The person closes windows",
                                "premises": [
                                    {{
                                        "id": "C2.1",
                                        "text": "Closing windows prevents cold air from entering the room."
                                    }},
                                    {{
                                        "id": "C2.2",
                                        "text": "Closing windows prevents heat from escaping through drafts."
                                    }}
                                ],
                                "conclusion": "Therefore, closing windows effectively reduces external cold exposure."
                            }}
                        ]
                    }}
                ],
                "and_logic_evaluation": "A heater provides heat generation, AND closed windows prevent that heat from escaping. Both actions work together to maximize warmth.",
                "therefore": "This combination is the correct choice."
            }},
            {{
                "option": "D",
                "hypothesis": "The person stays warm",
                "sub_hypotheses": [
                    {{
                        "id": "H1",
                        "factor": "Internal heat is generated or retained",
                        "actions": [
                            {{
                                "id": "D1",
                                "description": "The person eats spicy food",
                                "premises": [
                                    {{
                                        "id": "D1.1",
                                        "text": "Spicy food triggers a sensation of warmth."
                                    }},
                                    {{
                                        "id": "D1.2",
                                        "text": "This sensation is due to nerve stimulation, not an increase in core body temperature."
                                    }}
                                ],
                                "conclusion": "Therefore, eating spicy food does not effectively help retain internal heat."
                            }}
                        ]
                    }},
                    {{
                        "id": "H2",
                        "factor": "Exposure to cold is reduced",
                        "actions": [
                            {{
                                "id": "D2",
                                "description": "The person takes a cold shower",
                                "premises": [
                                    {{
                                        "id": "D2.1",
                                        "text": "Cold showers expose the body to cold water."
                                    }},
                                    {{
                                        "id": "D2.2",
                                        "text": "Cold water causes rapid heat loss from the body."
                                    }}
                                ],
                                "conclusion": "Therefore, taking a cold shower significantly increases external cold exposure and causes major heat loss."
                            }}
                        ]
                    }}
                ],
                "and_logic_evaluation": "Eating spicy food provides a minimal, psychological sense of warmth, BUT a cold shower causes significant heat loss.",
                "therefore": "This combination is counterproductive and not the best choice."
            }}
        ],
        "final_answer": "C"
    }}
    
    Now your turn:
    Question:
    {str(row['question'])}
    
    Options:
    A. {str(row['A'])}
    B. {str(row['B'])}
    C. {str(row['C'])}
    D. {str(row['D'])}
    
    Generate the response as a JSON object, following the exact example format.
    Output: {{
      "output_breakdown": [
        {{
          "option": "A",
          "hypothesis": "...",
          "sub_hypotheses": [
            {{
              "id": "H1",
              "factor": "...",
              "actions": [
                {{
                  "id": "A1",
                  "description": "...",
                  "premises": [
                    {{
                      "id": "A1.1",
                      "text": "..."
                    }},
                    {{
                      "id": "A1.2",
                      "text": "..."
                    }}
                  ],
                  "conclusion": "..."
                }}
              ]
            }},
            {{
              "id": "H2",
              "factor": "...",
              "actions": [
                {{
                  "id": "A2",
                  "description": "...",
                  "premises": [
                    {{
                      "id": "A2.1",
                      "text": "..."
                    }},
                    {{
                      "id": "A2.2",
                      "text": "..."
                    }}
                  ],
                  "conclusion": "..."
                }}
              ]
            }}
          ],
          "and_logic_evaluation": "...",
          "therefore": "..."
        }},
        {{
          "option": "B",
          "hypothesis": "...",
          "sub_hypotheses": [
            {{
              "id": "H1",
              "factor": "...",
              "actions": [
                {{
                  "id": "B1",
                  "description": "...",
                  "premises": [
                    {{
                      "id": "B1.1",
                      "text": "..."
                    }},
                    {{
                      "id": "B1.2",
                      "text": "..."
                    }}
                  ],
                  "conclusion": "..."
                }}
              ]
            }},
            {{
              "id": "H2",
              "factor": "...",
              "actions": [
                {{
                  "id": "B2",
                  "description": "...",
                  "premises": [
                    {{
                      "id": "B2.1",
                      "text": "..."
                    }},
                    {{
                      "id": "B2.2",
                      "text": "..."
                    }}
                  ],
                  "conclusion": "..."
                }}
              ]
            }}
          ],
          "and_logic_evaluation": "...",
          "therefore": "..."
        }},
        {{
          "option": "C",
          "hypothesis": "...",
          "sub_hypotheses": [
            {{
              "id": "H1",
              "factor": "...",
              "actions": [
                {{
                  "id": "C1",
                  "description": "...",
                  "premises": [
                    {{
                      "id": "C1.1",
                      "text": "..."
                    }},
                    {{
                      "id": "C1.2",
                      "text": "..."
                    }}
                  ],
                  "conclusion": "..."
                }}
              ]
            }},
            {{
              "id": "H2",
              "factor": "...",
              "actions": [
                {{
                  "id": "C2",
                  "description": "...",
                  "premises": [
                    {{
                      "id": "C2.1",
                      "text": "..."
                    }},
                    {{
                      "id": "C2.2",
                      "text": "..."
                    }}
                  ],
                  "conclusion": "..."
                }}
              ]
            }}
          ],
          "and_logic_evaluation": "...",
          "therefore": "..."
        }},
        {{
          "option": "D",
          "hypothesis": "...",
          "sub_hypotheses": [
            {{
              "id": "H1",
              "factor": "...",
              "actions": [
                {{
                  "id": "D1",
                  "description": "...",
                  "premises": [
                    {{
                      "id": "D1.1",
                      "text": "..."
                    }},
                    {{
                      "id": "D1.2",
                      "text": "..."
                    }}
                  ],
                  "conclusion": "..."
                }}
              ]
            }},
            {{
              "id": "H2",
              "factor": "...",
              "actions": [
                {{
                  "id": "D2",
                  "description": "...",
                  "premises": [
                    {{
                      "id": "D2.1",
                      "text": "..."
                    }},
                    {{
                      "id": "D2.2",
                      "text": "..."
                    }}
                  ],
                  "conclusion": "..."
                }}
              ]
            }}
          ],
          "and_logic_evaluation": "...",
          "therefore": "..."
        }}
      ],
      "final_answer": ""
    }}
    """
    
    messages = [
    {"role": "system", "content": "You are a logical reasoning AI designed to evaluate complex, multi-factor, multiple-choice questions. To find the correct answer, you must analyze each option and determine if both parts of the 'AND' combination positively contribute to the overall goal of the question. Your task is to analyze each option and provide a structured, multi-step breakdown."},
    {"role": "user", "content": user_content}
    ]
    return messages


def process_single_row(qa_df, output_file_csv):
    """
    Orchestrates the single-stage pipeline and saves the final output to a CSV file.
    """
    print(f"Starting single-stage pipeline for {len(qa_df)} rows.")
    final_output_list = []
    
    required_cols = ['question', 'A', 'B', 'C', 'D', 'correct_label']
    if not all(col in qa_df.columns for col in required_cols):
        raise ValueError(f"DataFrame must contain the following columns: {required_cols}")

    valid_choices = {'A', 'B', 'C', 'D'}

    for idx, row in qa_df.iterrows():
        # This is where the model generates the question
        messages = one_shot_two_depth_reasoning(row)
        try:
            # Call the API and get the full JSON output
            api_result_dict = call_gpt4o_mini(messages)
            
            # --- POST-PROCESSING LOGIC HERE ---
            # Extract the reasoning and final answer
            generated_reasoning = json.dumps(api_result_dict.get('output_breakdown', []))
            predicted_label = api_result_dict.get('final_answer')
            
            # Create the final result dictionary for this row
            result = {
                'question': row['question'],
                'options': f"A. {row['A']} B. {row['B']} C. {row['C']} D. {row['D']}",
                'reasoning_full': generated_reasoning,
                'predicted_label': predicted_label.strip().upper() if predicted_label and predicted_label.strip().upper() in valid_choices else None,
                'correct_label': row['correct_label']
            }
            # --- END OF POST-PROCESSING LOGIC ---

            final_output_list.append(result)
            print(f"Processed row {idx} successfully. Predicted: {result.get('predicted_label')}, Correct: {result.get('correct_label')}")
            
        except Exception as e:
            print(f"Failed to process row {idx}. Error: {e}. Skipping.")
            final_output_list.append({
                'question': row['question'],
                'options': f"A. {row['A']} B. {row['B']} C. {row['C']} D. {row['D']}",
                'reasoning_full': "",
                'predicted_label': None,
                'correct_label': row['correct_label']
            })

    final_df = pd.DataFrame(final_output_list)
    final_df.to_csv(output_file_csv, index=False, encoding='utf-8')
    print(f"\nProcessing complete. {len(final_output_list)} final results saved to '{output_file_csv}'.")

# --- Example of running the pipeline ---
if __name__ == '__main__':
    process_single_row(qa_df.iloc[:200], 'csqa_oneshot_2d_reasoning.csv')


Starting single-stage pipeline for 200 rows.
Processed row 0 successfully. Predicted: A, Correct: A
Processed row 1 successfully. Predicted: B, Correct: B
Processed row 2 successfully. Predicted: None, Correct: C
Processed row 3 successfully. Predicted: B, Correct: D
Processed row 4 successfully. Predicted: A, Correct: A
Processed row 5 successfully. Predicted: B, Correct: B
Processed row 6 successfully. Predicted: C, Correct: C
Processed row 7 successfully. Predicted: D, Correct: D
Processed row 8 successfully. Predicted: A, Correct: A
Processed row 9 successfully. Predicted: B, Correct: B
Processed row 10 successfully. Predicted: C, Correct: C
Processed row 11 successfully. Predicted: B, Correct: D
Processed row 12 successfully. Predicted: D, Correct: A
Processed row 13 successfully. Predicted: B, Correct: B
Processed row 14 successfully. Predicted: C, Correct: C
Processed row 15 successfully. Predicted: D, Correct: D
Processed row 16 successfully. Predicted: D, Correct: A
Processed 

In [15]:
# import pandas as pd
# import json

# # Replace 'csqa_oneshot_1d_reasoning.csv' with the name of your output file
# output_file_csv = 'csqa_oneshot_1d_reasoning.csv'

# # Load the CSV file into a DataFrame
# try:
#     results_df = pd.read_csv(output_file_csv)
# except FileNotFoundError:
#     print(f"Error: The file '{output_file_csv}' was not found.")
#     exit()

# # Iterate through the DataFrame and print the desired columns
# print(f"Displaying results from '{output_file_csv}':\n")

# for index, row in results_df.iterrows():
#     print(f"Q: {row['question']}")
#     print(f"Options: {row['options']}")
#     print(f"Correct answer: {row['correct_label']}")
#     print(f"Predicted answer: {row['predicted_label']}")
    
#     # The reasoning column contains a JSON string, so we need to parse it
#     # to print it nicely, otherwise it will just be a long string
#     try:
#         reasoning_data = json.loads(row['reasoning_full'])
#         print(f"Reasoning:")
#         for option_breakdown in reasoning_data:
#             option = option_breakdown['option']
#             evaluation = option_breakdown['and_logic_evaluation']
#             print(f"  Option {option}: {evaluation}")
#     except json.JSONDecodeError:
#         print(f"Reasoning: (Could not parse JSON for this row)")
#         print(row['reasoning_full'])
        
#     print("-" * 20)

output_file_csv = 'csqa_oneshot_2d_reasoning.csv'

# Load the CSV file
try:
    results_df = pd.read_csv(output_file_csv)
except FileNotFoundError:
    print(f"Error: The file '{output_file_csv}' was not found.")
    exit()

# Iterate through all rows and print the formatted JSON output for each
print(f"Displaying formatted JSON output for all rows from '{output_file_csv}':\n")

for index, row in results_df.iloc[:25].iterrows():
    # Print a header for the current question
    print(f"--- Question {index + 1} ---")
    print(f"Question: {row['question']}")
    print(f"Correct Answer: {row['correct_label']}")
    print(f"Predicted Answer: {row['predicted_label']}\n")

    # Get the JSON string and pretty-print it
    try:
        json_string = row['reasoning_full']
        reasoning_data = json.loads(json_string)
        
        # Use json.dumps with indent=2 for a clean, hierarchical output
        pretty_json = json.dumps(reasoning_data, indent=2)
        print("Reasoning Breakdown (JSON):")
        print(pretty_json)
        
    except (json.JSONDecodeError, KeyError) as e:
        print(f"Error processing JSON for this row: {e}")
        print("Raw Data:")
        print(row['reasoning_full'])
    
    print("-" * 50 + "\n")

Displaying formatted JSON output for all rows from 'csqa_oneshot_2d_reasoning.csv':

--- Question 1 ---
Question: The sanctions against the school were a punishing blow, and they seemed to what the efforts the school had made to change?
Correct Answer: A
Predicted Answer: A

Reasoning Breakdown (JSON):
[
  {
    "option": "A",
    "hypothesis": "The sanctions against the school negate the efforts made to change.",
    "sub_hypotheses": [
      {
        "id": "H1",
        "factor": "The efforts made by the school are disregarded.",
        "actions": [
          {
            "id": "A1",
            "description": "The sanctions ignore the efforts made by the school.",
            "premises": [
              {
                "id": "A1.1",
                "text": "Ignoring means not acknowledging or considering something."
              },
              {
                "id": "A1.2",
                "text": "If the sanctions ignore the efforts, it means they do not recognize any posi